In [1]:
import findspark
findspark.init("/home/raj/spark/")


In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SparkSession

In [7]:
def DecisionTree():
    

    spark = SparkSession.builder.appName("DecisionTreeRegressionExample").getOrCreate()

 
    dataset = spark.read.format("csv").option("inferSchema","true").option("header","true").load("/home/raj/Downloads/notenook/adult2.csv")

#dataset.printSchema()



    
    #indexer converts categorical data into numerics 
    workclass_stringIndexer = StringIndexer(inputCol="workclass", outputCol="workclass_indexed")
    si_model_workclass = workclass_stringIndexer.fit(dataset)

    education_stringIndexer = StringIndexer(inputCol="education", outputCol="education_indexed")
    si_model_education = education_stringIndexer.fit(dataset)

    occupation_stringIndexer = StringIndexer(inputCol="occupation", outputCol="occupation_indexed")
    si_model_occupation = occupation_stringIndexer.fit(dataset)

    sex_stringIndexer = StringIndexer(inputCol="sex", outputCol="sex_indexed")
    si_model_sex = sex_stringIndexer.fit(dataset)

    income_stringIndexer = StringIndexer(inputCol="income", outputCol="income_indexed")
    si_model_income = income_stringIndexer.fit(dataset)

    td = si_model_workclass.transform(si_model_education.transform(si_model_occupation.transform(si_model_sex.transform(si_model_income.transform(dataset)))))

#convert to pandas dataframe
    td.toPandas()



    feature_columns = ["age","workclass_indexed","fnlwgt","education_num","hours_per_week","sex_indexed","education_indexed","occupation_indexed"]

    feature_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    data = feature_assembler.transform(td)

    # Automatically identify categorical features, and index them.
    # We specify maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

    # Split the data into training and test sets (30% held out for testing)
    (trainingData, testData) = data.randomSplit([0.7, 0.3])
    print(featureIndexer)

    # Train a DecisionTree model.
    dt = DecisionTreeRegressor(featuresCol="indexedFeatures",labelCol="income_indexed")

    # Chain indexer and tree in a Pipeline
    pipeline = Pipeline(stages=[featureIndexer, dt])

    # Train model.  This also runs the indexer.
    model = pipeline.fit(trainingData)

    # Make predictions.
    predictions = model.transform(testData)

    # Select example rows to display.
    predictions.select("prediction", "income_indexed", "features").show(5)

    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(
       labelCol="income_indexed", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

    treeModel = model.stages[1]
    
    return rmse , treeModel



In [8]:
DecisionTree()

VectorIndexer_4a8099a79d75b2a89f75
+--------------------+--------------+--------------------+
|          prediction|income_indexed|            features|
+--------------------+--------------+--------------------+
|0.023809523809523808|           0.0|[17.0,3.0,80077.0...|
|0.023809523809523808|           0.0|[17.0,3.0,202521....|
|                 0.0|           0.0|[17.0,2.0,32124.0...|
|                 0.0|           0.0|[17.0,0.0,65368.0...|
|                 0.0|           0.0|[17.0,0.0,100828....|
+--------------------+--------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.390651


(0.39065096570689944,
 DecisionTreeRegressionModel (uid=DecisionTreeRegressor_411db34f11fa339084ac) of depth 5 with 59 nodes)